In [1]:
import cv2
import numpy as np
import time

In [2]:
cnt_up   = 0
cnt_down = 0

In [12]:
cap = cv2.VideoCapture('jacob4.mp4')

In [13]:
fgbg = cv2.createBackgroundSubtractorMOG2(detectShadows = True) #Create the background substractor

In [14]:
kernelOp = np.ones((3,3),np.uint8)
kernelCl = np.ones((11,11),np.uint8)

In [15]:
font = cv2.FONT_HERSHEY_SIMPLEX
persons = []
max_p_age = 5
pid = 1
areaTH = 500

In [16]:
w = cap.get(3)
h = cap.get(4)
frameArea = h*w
areaTH = frameArea/250
print ('Area Threshold', areaTH)

Area Threshold 737.28


In [17]:
line_up = int(2*(h/5))
line_down   = int(3*(h/5))

up_limit =   int(1*(h/5))
down_limit = int(4*(h/5))

In [18]:
print ("Red line y:", str(line_down))
print ("Blue line y:", str(line_up))
line_down_color = (255,0,0)
line_up_color = (0,0,255)
pt1 =  [0, line_down];
pt2 =  [w, line_down];
pts_L1 = np.array([pt1,pt2], np.int32)
pts_L1 = pts_L1.reshape((-1,1,2))
pt3 =  [0, line_up];
pt4 =  [w, line_up];
pts_L2 = np.array([pt3,pt4], np.int32)
pts_L2 = pts_L2.reshape((-1,1,2))

pt5 =  [0, up_limit];
pt6 =  [w, up_limit];
pts_L3 = np.array([pt5,pt6], np.int32)
pts_L3 = pts_L3.reshape((-1,1,2))
pt7 =  [0, down_limit];
pt8 =  [w, down_limit];
pts_L4 = np.array([pt7,pt8], np.int32)
pts_L4 = pts_L4.reshape((-1,1,2))

Red line y: 345
Blue line y: 230


In [19]:
class Persons:
    def __init__(self, pid, x, y, maxpage):
        self.id = pid
        self.x = x
        self.y = y
        self.maxpage = maxpage
        self.data = []
        self.age = 0
        self.state = 0
        self.direction = "none"
        self.done = 0
    def getTracks(self):
        return self.data
    def getId(self):
        return self.id
    def getX(self):
        return self.x
    def getY(self):
        return self.y
    def updateCoords(self, a, b):
        self.data.append([self.x, self.y])
        self.x = a
        self.y = b
    def age_one(self):
        self.age += 1
    def going_UP(self, linedown, lineup):
        if (self.y < linedown and self.data[-1][1] > linedown): 
            self.direction = "up"
            self.state = 1
            return True
        else: return False
    def going_DOWN(self, linedown, lineup):
        if (self.y > lineup and self.data[-1][1] < lineup): 
            self.direction = "down"
            self.state = 1
            return True
        else: return False
    def getState(self):
        return self.state
    def getDir(self):
        return self.direction
    def setDone(self):
        self.done = 1
    def timedOut(self):
        if (self.age > 10): return True
        elif (self.done == 1): return True
        else: return False

In [20]:
while(cap.isOpened()):
    ret, frame = cap.read()
    for i in persons:
        i.age_one() #age every person one frame

    fgmask = fgbg.apply(frame)
    fgmask2 = fgbg.apply(frame)
    try:
        ret,imBin= cv2.threshold(fgmask,200,255,cv2.THRESH_BINARY)
        ret,imBin2 = cv2.threshold(fgmask2,200,255,cv2.THRESH_BINARY)
        mask = cv2.morphologyEx(imBin, cv2.MORPH_OPEN, kernelOp)
        mask2 = cv2.morphologyEx(imBin2, cv2.MORPH_OPEN, kernelOp)
        mask =  cv2.morphologyEx(mask , cv2.MORPH_CLOSE, kernelCl)
        mask2 = cv2.morphologyEx(mask2, cv2.MORPH_CLOSE, kernelCl)
    except:
        print('EOF')
        print('UP:',cnt_up)
        print('DOWN:',cnt_down)
        break

    _, contours0, hierarchy = cv2.findContours(mask2,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    for cnt in contours0:
        area = cv2.contourArea(cnt)
        if area > areaTH:
            M = cv2.moments(cnt)
            cx = int(M['m10']/M['m00'])
            cy = int(M['m01']/M['m00'])
            x,y,w,h = cv2.boundingRect(cnt)

            new = True
            if (cy in range(up_limit,down_limit)):
                for i in persons:
                    if (abs(cx-i.getX()) <= w and abs(cy-i.getY()) <= h):
                        new = False
                        i.updateCoords(cx,cy)   
                        if (i.going_UP(line_down,line_up) == True):
                            cnt_up += 1;
                            print ("ID:",i.getId(),'crossed going up at',time.strftime("%c"))
                        elif (i.going_DOWN(line_down,line_up) == True):
                            cnt_down += 1;
                            print ("ID:",i.getId(),'crossed going down at',time.strftime("%c"))
                        break
                    if (i.getState() == 1):
                        if (i.getDir() == 'down' and i.getY() > down_limit):
                            i.setDone()
                        elif (i.getDir() == 'up' and i.getY() < up_limit):
                            i.setDone()
                    if (i.timedOut()):
                        index = persons.index(i)
                        persons.pop(index)
                        del i     
                if (new == True):
                    p = Persons(pid, cx, cy, max_p_age)
                    persons.append(p)
                    pid += 1     

            cv2.circle(frame,(cx,cy), 5, (0,0,255), -1)
            img = cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)            

    for i in persons:
        cv2.putText(frame, str(i.getId()),(i.getX(),i.getY()),font,0.3,(0, 0, 255),1,cv2.LINE_AA)

    str_up = 'UP: '+ str(cnt_up)
    str_down = 'DOWN: '+ str(cnt_down)
    frame = cv2.polylines(frame,[pts_L1],False,line_down_color,thickness=2)
    frame = cv2.polylines(frame,[pts_L2],False,line_up_color,thickness=2)
    frame = cv2.polylines(frame,[pts_L3],False,(255,255,255),thickness=1)
    frame = cv2.polylines(frame,[pts_L4],False,(255,255,255),thickness=1)
    cv2.putText(frame, str_up ,(10,40),font,0.5,(255,255,255),2,cv2.LINE_AA)
    cv2.putText(frame, str_up ,(10,40),font,0.5,(0,0,255),1,cv2.LINE_AA)
    cv2.putText(frame, str_down ,(10,90),font,0.5,(255,255,255),2,cv2.LINE_AA)
    cv2.putText(frame, str_down ,(10,90),font,0.5,(255,0,0),1,cv2.LINE_AA)

    cv2.imshow('Frame',frame)

    k = cv2.waitKey(30) & 0xff
    if (k == 27): break

cap.release()
cv2.destroyAllWindows()

ID: 1 crossed going down at Sat Jul 14 18:11:07 2018
ID: 1 crossed going up at Sat Jul 14 18:11:12 2018
ID: 1 crossed going down at Sat Jul 14 18:11:18 2018
ID: 4 crossed going down at Sat Jul 14 18:11:33 2018
EOF
UP: 2
DOWN: 6
